In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
OBIPath = '/media/ak/Data/InterestRateFuturesData/ReconstructedLOB/OrderBookImbalance/'
OBIMFDFAResults = '/media/ak/Data/InterestRateFuturesData/ReconstructedLOB/OrderBookImbalance/OBIMFDFAResults'
TwoSampleDFs =  os.path.join(OBIMFDFAResults,'TwoSampleDataFrames')

In [3]:
os.listdir(TwoSampleDFs)

['listH',
 'alpha',
 'alphaMedianDFOBI.pkl',
 'Spectrum',
 'tau',
 'tauMedianDFOBI.pkl']

## Orange Drive Paths

In [4]:
# OBIPath = '/media/ak/LaCie/InterestRateFuturesData/ReconstructedLOB/OrderBookImbalance'

# OBIMFDFAResults = os.path.join(OBIPath, 'OBIMFDFAResults')

In [5]:
resultsPath = os.path.join(OBIMFDFAResults, 'TwoSampleDataFrames')

In [6]:
def process_dict(input_dict):
    """
    Process a given input dictionary to transform and restructure its data.

    This function iterates through each key-value pair in the input dictionary. 
    It processes the dictionary in the following ways:
    
    1. If the value is a list of dictionaries, each sub-dictionary is flattened. 
       For each key-value pair in the sub-dictionary, a new key is created in the 
       format "original_key_index_sub_key", and the sub-value is stored as a list.
       If the sub-value is a list or a numpy array, the median of the array is calculated 
       and stored in a list. If the median calculation raises a TypeError, the original 
       sub-value is stored as is.

    2. If the key is one of ['list_gwidth', 'list_kernels', 'list_permuted_mmd2'], 
       the value is stored directly without transformation.

    3. For other values, if they are lists or numpy arrays, the function tries to calculate 
       the median and store it in a list. If the median calculation raises a TypeError, 
       or if the array is empty, the original value is stored as is.

    4. For all other types of values, they are stored in a list.

    Parameters:
    - input_dict (dict): The input dictionary to be processed.

    Returns:
    - dict: A new dictionary with processed key-value pairs.
    """
    processed_dict = {}
    for key, value in input_dict.items():
        # Check if the value is a list of dictionaries
        if isinstance(value, list) and all(isinstance(item, dict) for item in value):
            for i, sub_dict in enumerate(value):
                for sub_key, sub_value in sub_dict.items():
                    new_key = f"{key}_{i}_{sub_key}"
                    # Process the sub_value similar to how other values are processed
                    if isinstance(sub_value, list) or (type(sub_value).__module__ == 'numpy' and isinstance(sub_value, np.ndarray)):
                        if len(sub_value) > 0:
                            try:
                                processed_dict[new_key] = [np.median(sub_value)]
                            except TypeError:
                                processed_dict[new_key] = sub_value
                        else:
                            processed_dict[new_key] = sub_value
                    else:
                        processed_dict[new_key] = [sub_value]
        elif key in ['list_gwidth', 'list_kernels', 'list_permuted_mmd2']:
            processed_dict[key] = value
        elif isinstance(value, list) or (type(value).__module__ == 'numpy' and isinstance(value, np.ndarray)):
            if len(value) > 0:
                try:
                    processed_dict[key] = [np.median(value)]
                except TypeError:
                    processed_dict[key] = value
            else:
                processed_dict[key] = value
        else:
            processed_dict[key] = [value]

    return processed_dict


In [7]:
# import pickle

# # Replace 'your_pickle_file.pkl' with the path to your pickle file
# pickle_file_path = filePath

# # Attempt to open the pickle file
# try:
#     with open(pickle_file_path, 'rb') as file:
#         data = pickle.load(file)
#     print("Pickle file loaded successfully.")
# except Exception as e:
#     print(f"An error occurred: {e}")


In [8]:
def proportion_second_value(final_df, column):
    total_count = final_df[column].count()
    if total_count > 0 and len(final_df[column].value_counts()) > 1:
        second_value_count = final_df[column].value_counts()[1]
        proportion = (second_value_count / total_count) * 100
    else:
        proportion = 0
    return f"{proportion:.2f}%"

# Calculate the proportion of the second value (which is 'True' in your case)
# true_proportion_XZ = proportion_second_value('XZ_test_h0_rejected')
# true_proportion_YZ = proportion_second_value('YZ_test_h0_rejected')

# # If you need to round numeric columns to two decimals
# median_values = final_df.median().round(3)

# # Store the formatted proportions in the median_values Series
# median_values['XZ_test_h0_rejected'] = true_proportion_XZ
# median_values['YZ_test_h0_rejected'] = true_proportion_YZ


In [9]:
def create_results_table(df):
    # format the columns 'pvalue_1', 'test_stat_1', 'pvalue_2', and 'test_stat_2'
    df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']] = df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']].applymap(lambda x: f'{float(x):.2e}'.replace('e-', 'x10^-') if pd.notna(x) else 'N/A')

    # replace the NaN values with 'N/A'
    df.fillna('N/A', inplace=True)

    # create the LaTeX table
    latex = df.to_latex(column_format='lcccccccc', escape=False, header=['', 'pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2', 'h0_rejected_1', 'Percentage', 'Median Window', 'Median Shift'], bold_rows=[0, len(df)-1], multicolumn_format='c')

    # return the LaTeX table
    return latex

In [10]:
def flatten_dict(d):
    def expand(key, value):
        if isinstance(value, dict):
            return [(str(key) + '_' + str(k), v) for k, v in flatten_dict(value).items()]
        elif isinstance(value, list) or (type(value).__module__ == 'numpy' and isinstance(value, np.ndarray)):
            if len(value) > 0:
                try:
                    return [(str(key), np.median(value))]
                except TypeError:
                    return [(str(key), value)]
            else:
                return [(str(key), value)]
        else:
            return [(str(key), value)]
    
    items = [item for k, v in d.items() for item in expand(k, v)]
    
    return dict(items)

# flattened_dict = flatten_dict(input_dict)


In [11]:
import pandas as pd

def process_and_flatten(file_path):
    """
    Process and flatten the contents of a pickle file.

    This function reads a pickle file specified by file_path, extracts
    and processes its contents (assuming a specific structure of the
    data), then flattens the dictionary structure into a single level. 
    Certain keys are removed from the final flattened dictionary as 
    part of data cleaning.

    Parameters:
    file_path (str): The path to the pickle file to be processed.

    Returns:
    dict: A flattened dictionary with specified keys removed.

    Note:
    - The function assumes the existence of 'process_dict' and 
      'flatten_dict' functions which are not defined in this snippet.
    - The structure of the pickle file and the dictionary keys are 
      assumed based on the function's implementation.
    """

    # Load the pickle file
    unPickledFile = pd.read_pickle(file_path)

    # Extract the first item's value from the dictionary
    resultsDict = unPickledFile[next(iter(unPickledFile))]

    # Process the dictionary (process_dict needs to be defined elsewhere)
    process_dict(resultsDict)

    # Flatten the dictionary (flatten_dict needs to be defined elsewhere)
    flattened_dict = flatten_dict(resultsDict)
    
    # List of keys to be removed from the flattened dictionary
    keys_to_remove = ['list_gwidth', 'list_kernels', 'best_ker']

    # Remove specified keys
    for key in keys_to_remove:
        flattened_dict.pop(key, None)
    
    return flattened_dict


In [12]:
symbolIDX = 3
symbols = os.listdir(OBIMFDFAResults)
symbolPath = os.path.join(OBIMFDFAResults, symbols[symbolIDX])
os.listdir(symbolPath)
keys = ['alpha', 'Spectrum', 'tau', 'Hlist']

In [13]:
keyIDX = 3
symbolKeyPath =  os.path.join(symbolPath, keys[keyIDX])
files = os.listdir(symbolKeyPath)


In [ ]:
# fileIdxPath 

In [ ]:
# print([f for f in enumerate(resultsKeys) ])

In [14]:

# fileIdx = 1
for fileIdx, _ in enumerate(files):
    fileIdxPath = os.path.join(symbolKeyPath, files[fileIdx])
    shift_str, window_str = files[fileIdx].split("_shift_")[1].split("_OBI")[0].split("_wind_")
    # Convert the extracted strings to integers (or floats if needed)
    shift = int(shift_str)  # Use float(shift_str) if the value can be a decimal
    window = int(window_str)  # Use float(window_str) if the value can be a decimal

    #####################################################
    unPickledFile = pd.read_pickle(fileIdxPath)
    idXkeys = list(unPickledFile.keys())
    resultsDict = unPickledFile[idXkeys[0]]
    resultsKeys = list(resultsDict.keys())
    process_dict(resultsDict)
    flattened_dict = flatten_dict(resultsDict)
    # List of keys to remove
    keys_to_remove = ['list_gwidth', 'list_kernels', 'best_ker']

    # Remove the specified keys from the flattened dictionary
    for key in keys_to_remove:
        flattened_dict.pop(key, None)  # Use pop to avoid KeyError if key is not present

    # Now you can convert this flattened dictionary into a DataFrame
    df = pd.DataFrame([flattened_dict])
    df['window']= window
    df['shift']= shift
    print(df)

   perm_mmds1  chi2_weights  sim_mmds      sig2       Kxy      mean       var  \
0   -0.002266  4.734486e-20 -0.001241  0.067401  0.939163  0.010414  0.000071   

        Kxx       Kyy  mean_gram  ...  XZ_test_test_stat  XZ_test_h0_rejected  \
0  0.937501  0.923127   0.010414  ...           0.061504                 True   

   XZ_test_list_permuted_mmd2  YZ_test_alpha  YZ_test_pvalue  \
0                   -0.002287           0.05             0.0   

   YZ_test_test_stat  YZ_test_h0_rejected  YZ_test_list_permuted_mmd2  window  \
0           0.149108                 True                   -0.002211      10   

   shift  
0      1  

[1 rows x 26 columns]
   perm_mmds1  chi2_weights  sim_mmds      sig2       Kxy      mean       var  \
0   -0.002266  4.734486e-20 -0.001241  0.067401  0.939163  0.010414  0.000071   

        Kxx       Kyy  mean_gram  ...  XZ_test_test_stat  XZ_test_h0_rejected  \
0  0.937501  0.923127   0.010414  ...          -0.007526                False   

   XZ_test_

In [15]:

# symbols = os.listdir(OBIMFDFAResults)
# for symbolIDX in range(0,13):
#     symbolPath = os.path.join(OBIMFDFAResults, symbols[symbolIDX])
#     os.listdir(symbolPath)
#     keys = ['alpha', 'Spectrum', 'tau', 'Hlist']
#     keyIDX = 3

#     symbolKeyPath =  os.path.join(symbolPath, keys[keyIDX])
#     files = os.listdir(symbolKeyPath)
#     # Using list comprehension to process each file and store the flattened dict
#     dfs = [pd.DataFrame([process_and_flatten(os.path.join(symbolKeyPath, file))])
#            .assign(**dict(zip(['window', 'shift'], map(int, file.split("_shift_")[1].split("_OBI")[0].split("_wind_")))))
#            for file in files]

#     # Concatenate all dataframes
#     final_df = pd.concat(dfs, ignore_index=True)

#     # final_df now contains all the data stacked together
#     # Assuming symbols, symbolIDX, keys, and keyIDX are already defined in your code
#     fileName = os.path.join(resultsPath,"_".join((symbols[symbolIDX], keys[keyIDX], "twoSampleResults.pkl")))

#     # Save the DataFrame to a pickle file
#     final_df.to_pickle(fileName)


In [16]:
#### Making Latex Tables 

In [70]:
resultsFiles = os.listdir(resultsPath)
# keys = ['tau', 'alpha', 'Spectrum', 'listH']
tauResultsFiles = os.path.join(resultsPath, 'tau')
alphaResultsFiles = os.path.join(resultsPath, 'alpha')
SpectrumResultsFiles = os.path.join(resultsPath, 'Spectrum')
hurstResultsFiles = os.path.join(resultsPath, 'listH')


In [18]:
os.listdir(resultsPath)

['listH',
 'alpha',
 'alphaMedianDFOBI.pkl',
 'Spectrum',
 'tau',
 'tauMedianDFOBI.pkl']

In [19]:
alphaFiles = os.listdir(alphaResultsFiles)

## alpha

In [52]:
list_dfs = []
for fileIdx in range(len(alphaFiles)):
    filePath = os.path.join(alphaResultsFiles, alphaFiles[fileIdx])
    list_dfs.append(pd.read_pickle(filePath))

alpha_median_df = pd.concat(list_dfs, axis=0).median(axis=0)
df1 = alpha_median_df.to_frame().T
df1['XZ_test_h0_rejected']=pd.concat(list_dfs, axis=1)['XZ_test_h0_rejected'].mean().mean()
df1['YZ_test_h0_rejected']=pd.concat(list_dfs, axis=1)['YZ_test_h0_rejected'].mean().mean()
alpha_median_path = os.path.join(TwoSampleDFs, 'alphaMedianDFOBI.pkl')
df1 = df1.T.rename(columns={0: 'alpha'})
df1.to_pickle(alpha_median_path )

## Tau

In [51]:
list_dfs_tau = []
tauFiles = os.listdir(tauResultsFiles)
for fileIdx in range(len(tauFiles)):

    filePath = os.path.join(tauResultsFiles, tauFiles[fileIdx])
    list_dfs_tau.append(pd.read_pickle(filePath))

tau_median_df = pd.concat(list_dfs_tau, axis=0).median(axis=0)
df2 = tau_median_df.to_frame().T
df2['XZ_test_h0_rejected']=pd.concat(list_dfs_tau, axis=1)['XZ_test_h0_rejected'].mean().mean()
df2['YZ_test_h0_rejected']=pd.concat(list_dfs_tau, axis=1)['YZ_test_h0_rejected'].mean().mean()
tau_median_path = os.path.join(TwoSampleDFs, 'tauMedianDFOBI.pkl')
df2 = df2.T.rename(columns={0: 'tau'})
df2.to_pickle(tau_median_path )

## Spectrum

In [61]:
list_dfs_spectrum = []
spectrumFiles = os.listdir(SpectrumResultsFiles)
for fileIdx in range(len(spectrumFiles)):

    filePath = os.path.join(SpectrumResultsFiles, spectrumFiles[fileIdx])
    list_dfs_spectrum.append(pd.read_pickle(filePath))

spectrum_median_df = pd.concat(list_dfs_spectrum, axis=0).median(axis=0)
df3 = spectrum_median_df.to_frame().T
df3['XZ_test_h0_rejected']=pd.concat(list_dfs_spectrum, axis=1)['XZ_test_h0_rejected'].mean().mean()
df3['YZ_test_h0_rejected']=pd.concat(list_dfs_spectrum, axis=1)['YZ_test_h0_rejected'].mean().mean()
spectrum_median_path = os.path.join(TwoSampleDFs, 'spectrumMedianDFOBI.pkl')
df3 = df3.T.rename(columns={0: 'spectrum'})
df3.to_pickle(spectrum_median_path)

## Hurst

In [80]:
list_dfs_hurst = []
hurstFiles = os.listdir(hurstResultsFiles)
for fileIdx in range(len(hurstFiles)):

    filePath = os.path.join(hurstResultsFiles, hurstFiles[fileIdx])
    list_dfs_hurst.append(pd.read_pickle(filePath))
hurst_median_df = pd.concat(list_dfs_hurst, axis=0).median(axis=0)
df4 = hurst_median_df.to_frame().T
df4['XZ_test_h0_rejected']=pd.concat(list_dfs_hurst, axis=1)['XZ_test_h0_rejected'].mean().mean()
df4['YZ_test_h0_rejected']=pd.concat(list_dfs_hurst, axis=1)['YZ_test_h0_rejected'].mean().mean()
hurst_median_path = os.path.join(TwoSampleDFs, 'hurstMedianDFOBI.pkl')
df4 = df4.T.rename(columns={0: 'Hurst'})
df4.to_pickle(hurst_median_path)

In [81]:
df4

,Hurst
perm_mmds1,-2.956847e-03
chi2_weights,1.393413e-19
sim_mmds,-1.743660e-03
sig2,2.557662e-01
Kxy,7.354095e-01
mean,1.830998e-01
var,1.365307e-03
Kxx,9.032252e-01
Kyy,8.861710e-01
mean_gram,1.830998e-01


In [ ]:
# alpha_median_df = pd.concat(list_dfs, axis=1).median(axis=0).T

In [82]:
consolidated = pd.concat([df1, df2, df3, df4], axis =1)

In [83]:
consolidated = consolidated.drop(['window', 'shift'], axis=0)

In [84]:
consolidated = consolidated.round(3)

In [ ]:

# # Assuming final_df is your DataFrame
# final_df= pd.concat(list_dfs_tau, axis=1)

# # Step 1: Calculate median for numeric columns
# median_values = final_df.median()

# # Step 2: Calculate proportion of 'True' for specified columns
# def true_proportion(df, column):
#     true_count = df[column].value_counts().get('True', 0)
#     total_count = df[column].count()
#     return true_count / total_count if total_count else 0

# true_proportion_XZ = true_proportion(final_df, 'XZ_test_h0_rejected')
# true_proportion_YZ = true_proportion(final_df, 'YZ_test_h0_rejected')

# # # Adding these proportions to the median_values Series for completeness
# # median_values['XZ_test_h0_rejected'] = true_proportion_XZ
# # median_values['YZ_test_h0_rejected'] = true_proportion_YZ

# # median_values now contains the medians of numeric columns and the proportions of 'True' for the specified columns


In [68]:
# df_for_latex = pd.DataFrame(median_values)

In [85]:
print(consolidated.to_latex(column_format='lcccccccc', escape=False, multicolumn_format='c'))

\begin{tabular}{lcccccccc}
\toprule
{} &  alpha &    tau &  spectrum &  Hurst \\
\midrule
perm_mmds1                 & -0.004 & -0.003 &    -0.003 & -0.003 \\
chi2_weights               &  0.000 &  0.000 &     0.000 &  0.000 \\
sim_mmds                   & -0.002 & -0.002 &    -0.002 & -0.002 \\
sig2                       &  0.351 &  2.711 &     0.215 &  0.256 \\
Kxy                        &  0.642 &  0.068 &     0.804 &  0.735 \\
mean                       &  0.100 &  0.014 &     0.039 &  0.183 \\
var                        &  0.000 &  0.000 &     0.000 &  0.001 \\
Kxx                        &  0.803 &  0.078 &     0.863 &  0.903 \\
Kyy                        &  0.811 &  0.077 &     0.840 &  0.886 \\
mean_gram                  &  0.100 &  0.014 &     0.039 &  0.183 \\
var_gram                   &  0.000 &  0.000 &     0.000 &  0.001 \\
med                        &  0.351 &  2.711 &     0.215 &  0.256 \\
besti                      &  3.000 &  3.500 &     7.500 &  4.500 \\
powers       

In [86]:
consolidated

,alpha,tau,spectrum,Hurst
perm_mmds1,-0.004,-0.003,-0.003,-0.003
chi2_weights,0.000,0.000,0.000,0.000
sim_mmds,-0.002,-0.002,-0.002,-0.002
sig2,0.351,2.711,0.215,0.256
Kxy,0.642,0.068,0.804,0.735
mean,0.100,0.014,0.039,0.183
var,0.000,0.000,0.000,0.001
Kxx,0.803,0.078,0.863,0.903
Kyy,0.811,0.077,0.840,0.886
mean_gram,0.100,0.014,0.039,0.183


In [87]:
pvalue_rows = consolidated.filter(like='_pvalue', axis=0)

In [89]:
pvalue_rows.mean()

alpha       0.0000
tau         0.4535
spectrum    0.0055
Hurst       0.0000
dtype: float64

In [90]:
# Calculate the mean of the rows containing '_pvalue'
pvalue_mean = consolidated.filter(like='_pvalue', axis=0).mean()

# Remove the original '_pvalue' rows
consolidated = consolidated.drop(index=consolidated.filter(like='_pvalue', axis=0).index)

# Add a new row with the name "Median p-value" and the calculated mean values
consolidated.loc["Median p-value"] = pvalue_mean

# Print the updated DataFrame
print(consolidated)

                            alpha     tau  spectrum  Hurst
perm_mmds1                 -0.004 -0.0030   -0.0030 -0.003
chi2_weights                0.000  0.0000    0.0000  0.000
sim_mmds                   -0.002 -0.0020   -0.0020 -0.002
sig2                        0.351  2.7110    0.2150  0.256
Kxy                         0.642  0.0680    0.8040  0.735
mean                        0.100  0.0140    0.0390  0.183
var                         0.000  0.0000    0.0000  0.001
Kxx                         0.803  0.0780    0.8630  0.903
Kyy                         0.811  0.0770    0.8400  0.886
mean_gram                   0.100  0.0140    0.0390  0.183
var_gram                    0.000  0.0000    0.0000  0.001
med                         0.351  2.7110    0.2150  0.256
besti                       3.000  3.5000    7.5000  4.500
powers                      3.021  0.3430    1.9510  3.890
XZ_test_alpha               0.050  0.0500    0.0500  0.050
XZ_test_test_stat           0.231 -0.0010    0.0610  0.2

In [91]:
import pandas as pd

# Assuming 'consolidated' is your DataFrame
# Example DataFrame creation:
# consolidated = pd.DataFrame({
#     "Column1": [1, 2, 3, 4, 5, 6, 7],
#     "Column2": [8, 9, 10, 11, 12, 13, 14],
# }, index=["row1", "row2_permuted_mmd2", "row3_test_stat", "row4_h0_rejected", "row5_permuted_mmd2", "row6_test_stat", "row7_h0_rejected"])

# Function to replace specific rows with their mean
def replace_rows_with_mean(df, substring, new_row_name):
    mean_values = df.filter(like=substring, axis=0).mean()
    df = df.drop(index=df.filter(like=substring, axis=0).index)
    df.loc[new_row_name] = mean_values
    return df

# Replace rows containing 'permuted_mmd2', 'test_stat', 'h0_rejected' with their means
consolidated = replace_rows_with_mean(consolidated, 'permuted_mmd2', 'Median permuted_mmd2')
consolidated = replace_rows_with_mean(consolidated, 'test_stat', 'Median test_stat')
consolidated = replace_rows_with_mean(consolidated, 'h0_rejected', 'Median h0_rejected')

# Print the updated DataFrame
print(consolidated)


                       alpha     tau  spectrum   Hurst
perm_mmds1           -0.0040 -0.0030   -0.0030 -0.0030
chi2_weights          0.0000  0.0000    0.0000  0.0000
sim_mmds             -0.0020 -0.0020   -0.0020 -0.0020
sig2                  0.3510  2.7110    0.2150  0.2560
Kxy                   0.6420  0.0680    0.8040  0.7350
mean                  0.1000  0.0140    0.0390  0.1830
var                   0.0000  0.0000    0.0000  0.0010
Kxx                   0.8030  0.0780    0.8630  0.9030
Kyy                   0.8110  0.0770    0.8400  0.8860
mean_gram             0.1000  0.0140    0.0390  0.1830
var_gram              0.0000  0.0000    0.0000  0.0010
med                   0.3510  2.7110    0.2150  0.2560
besti                 3.0000  3.5000    7.5000  4.5000
powers                3.0210  0.3430    1.9510  3.8900
XZ_test_alpha         0.0500  0.0500    0.0500  0.0500
YZ_test_alpha         0.0500  0.0500    0.0500  0.0500
Median p-value        0.0000  0.4535    0.0055  0.0000
Median per

In [96]:
print(consolidated.round(3).to_latex(column_format='lcccccccc', escape=False, multicolumn_format='c'))

\begin{tabular}{lcccccccc}
\toprule
{} &  alpha &    tau &  spectrum &  Hurst \\
\midrule
perm_mmds1           & -0.004 & -0.003 &    -0.003 & -0.003 \\
chi2_weights         &  0.000 &  0.000 &     0.000 &  0.000 \\
sim_mmds             & -0.002 & -0.002 &    -0.002 & -0.002 \\
sig2                 &  0.351 &  2.711 &     0.215 &  0.256 \\
Kxy                  &  0.642 &  0.068 &     0.804 &  0.735 \\
mean                 &  0.100 &  0.014 &     0.039 &  0.183 \\
var                  &  0.000 &  0.000 &     0.000 &  0.001 \\
Kxx                  &  0.803 &  0.078 &     0.863 &  0.903 \\
Kyy                  &  0.811 &  0.077 &     0.840 &  0.886 \\
mean_gram            &  0.100 &  0.014 &     0.039 &  0.183 \\
var_gram             &  0.000 &  0.000 &     0.000 &  0.001 \\
med                  &  0.351 &  2.711 &     0.215 &  0.256 \\
besti                &  3.000 &  3.500 &     7.500 &  4.500 \\
powers               &  3.021 &  0.343 &     1.951 &  3.890 \\
XZ_test_alpha        &  0.05

In [97]:
print(consolidated.round(2).to_latex(column_format='lcccccccc', escape=False, multicolumn_format='c'))

\begin{tabular}{lcccccccc}
\toprule
{} &  alpha &   tau &  spectrum &  Hurst \\
\midrule
perm_mmds1           &  -0.00 & -0.00 &     -0.00 &  -0.00 \\
chi2_weights         &   0.00 &  0.00 &      0.00 &   0.00 \\
sim_mmds             &  -0.00 & -0.00 &     -0.00 &  -0.00 \\
sig2                 &   0.35 &  2.71 &      0.22 &   0.26 \\
Kxy                  &   0.64 &  0.07 &      0.80 &   0.74 \\
mean                 &   0.10 &  0.01 &      0.04 &   0.18 \\
var                  &   0.00 &  0.00 &      0.00 &   0.00 \\
Kxx                  &   0.80 &  0.08 &      0.86 &   0.90 \\
Kyy                  &   0.81 &  0.08 &      0.84 &   0.89 \\
mean_gram            &   0.10 &  0.01 &      0.04 &   0.18 \\
var_gram             &   0.00 &  0.00 &      0.00 &   0.00 \\
med                  &   0.35 &  2.71 &      0.22 &   0.26 \\
besti                &   3.00 &  3.50 &      7.50 &   4.50 \\
powers               &   3.02 &  0.34 &      1.95 &   3.89 \\
XZ_test_alpha        &   0.05 &  0.05 &    